In [1]:
import pinocchio as pin
from pinocchio.utils import *

import example_robot_data as robex
import numpy as np
from numpy.linalg import inv, pinv, eig, norm, svd, det
from scipy.optimize import fmin_bfgs
import time
import copy
import hppfcl
import vizutils



In [2]:
robot = robex.loadSolo()

#Viewer = pin.visualize.GepettoVisualizer
Viewer = pin.visualize.MeshcatVisualizer

viz = Viewer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(loadModel=True)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
hasattr(viz.viewer, 'jupyter_cell') and viz.viewer.jupyter_cell()


In [4]:
viz.display(robot.q0)
xi = robot.q0[:3]

vizutils.addViewerBox(viz, 'world/box', .2, .5, .4, [1., 1, .2, 1])
vizutils.applyViewerConfiguration(viz, 'world/box', [1, -.2, .2, 1, 0, 0, 0])

xf = [1.7, .2, .2, 1, 0, 0, 0]  # x, y, z, quaternion
vizutils.addViewerSphere(viz, 'world/end', .04, [.2, .2, 1., .5])
vizutils.applyViewerConfiguration(viz, 'world/end', xf)

xstep = [-.05, .1, 0, 1, 0, 0, 0]
vizutils.addViewerSphere(viz, 'world/ball', .02, [.2, 1, .1, .5])
vizutils.applyViewerConfiguration(viz, 'world/ball', xstep)

In [22]:
class RootBox:
    def __init__(self):
        vizutils.addViewerBox(viz, 'world/root', .43, .26, .15, [1., .2, .2, .5])
        
    def alpha(self,alph):
        self.alph = alph + 0.0
        vizutils.addViewerBox(viz, 'world/root', .43, .26, .15, [1., .2, .2, alph])

        
    def move(self, q):
        xRoot = q.copy()[:7]
        vizutils.applyViewerConfiguration(viz, 'world/root', xRoot)
        
rootBox = RootBox()

In [25]:
rootBox.alpha(0)

In [7]:
rmodel = robot.model
rdata = rmodel.createData()

for f in robot.model.frames:
    print(f.name)

universe
root_joint
base_link
FL_HFE
FL_UPPER_LEG
FL_KFE
FL_LOWER_LEG
FL_ANKLE
FL_FOOT
FR_HFE
FR_UPPER_LEG
FR_KFE
FR_LOWER_LEG
FR_ANKLE
FR_FOOT
HL_HFE
HL_UPPER_LEG
HL_KFE
HL_LOWER_LEG
HL_ANKLE
HL_FOOT
HR_HFE
HR_UPPER_LEG
HR_KFE
HR_LOWER_LEG
HR_ANKLE
HR_FOOT


In [8]:
# Get index of end effector
idx = rmodel.getFrameId('HL_FOOT')
print(idx)



20


### GHOST-ROBODOG:

In [30]:
class Cost3d:
    def __init__(self, rmodel, rdata, frameIndex=None, ptarget=None, viz=None):
        self.rmodel = rmodel
        self.rdata = rdata
        self.ptarget = ptarget if ptarget is not None else  np.array([0.5, 0.1, 0.27])
        self.frameIndex = frameIndex if frameIndex is not None else robot.model.nframes - 1
        self.viz = viz

    def residual(self, q):
        pin.framesForwardKinematics(self.rmodel, self.rdata, q)
        M = self.rdata.oMf[self.frameIndex]
        return M.translation - self.ptarget

    def calc(self, q):
        return sum(self.residual(q) ** 2)

    # --- Callback
    def callback(self, q):
        if self.viz is None:
            return
        pin.framesForwardKinematics(self.rmodel, self.rdata, q)
        M = self.rdata.oMf[self.frameIndex]
        rootBox.move(q)
        #vizutils.applyViewerConfiguration(self.viz, 'world/ball', pin.SE3ToXYZQUATtuple(M))
        #vizutils.applyViewerConfiguration(self.viz, 'world/box', self.ptarget.tolist() + [1, 0, 0, 0])
        self.viz.display(q)
        time.sleep(5)

In [38]:
cost = Cost3d(rmodel, rdata, frameIndex = 1, ptarget= xf[:3], viz=viz)
qguess = robot.q0.copy()
qopt = fmin_bfgs(cost.calc, qguess, callback=cost.callback)

rootBox.move(qopt)
viz.display(qopt)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 51
         Gradient evaluations: 3


In [37]:
cost = Cost3d(rmodel, rdata, frameIndex = 1, ptarget= xi, viz=viz)
qguess = robot.q0.copy()
qopt = fmin_bfgs(cost.calc, qguess, callback=cost.callback)

rootBox.move(qopt)
viz.display(qopt)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 17
         Gradient evaluations: 1


### Pasito

In [19]:
cost = Cost3d(rmodel, rdata, frameIndex = 20, ptarget= xstep[:3], viz=viz)
#qguess = robot.q0.copy()
qopt = fmin_bfgs(cost.calc, qguess, callback=cost.callback)
viz.display(qopt)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 5
         Function evaluations: 153
         Gradient evaluations: 9


In [13]:
class CostPosture:
    def __init__(self, rmodel, rdata, qref=None, viz=None):
        # viz, rmodel and rdata are taken to respect the API but are not useful.
        self.qref = qref if qref is not None else pin.randomConfiguration(rmodel)
        self.removeFreeFlyer = robot.model.joints[1].nq == 7  # Don't penalize the free flyer if any.

    def residual(self, q):
        if self.removeFreeFlyer:
            return (q - self.qref)[7:]
        else:
            return q - self.qref

    def calc(self, q):
        return sum(self.residual(q) ** 2)

In [14]:
class SumOfCostFreeFlyer:
    def __init__(self, rmodel, rdata, costs, weights):
        self.rmodel = rmodel
        self.costs = costs
        self.weights = np.array(weights)

    def calc(self, q):
        q = pin.normalize(self.rmodel, q)
        # Alternatively, add this extra term to the sum
        extra = sum((q - pin.normalize(rmodel, q)) ** 2) * 100
        return sum(self.weights * [cost.calc(q) for cost in self.costs])   + extra

    def callback(self, q):
        q = pin.normalize(self.rmodel, q)
        rootBox.move(q)
        for c in self.costs:
            if hasattr(c, 'callback'):
                c.callback(q)
       
        time.sleep(0.05)


In [35]:
rootBox.alpha(0.5)

In [36]:
import random
randX = random.uniform(-4.1, 4.1)
randY = random.uniform(-4.1, 4.1)



mycost = SumOfCostFreeFlyer(rmodel, rdata,
                            [Cost3d(rmodel, rdata, frameIndex = 1, ptarget=[randX,randY,.2], viz=viz),
                             CostPosture(rmodel, rdata,qref = qguess )], [1, 1e-3])
qopt = fmin_bfgs(mycost.calc, robot.q0, callback=mycost.callback)
viz.display(qopt)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 51
         Gradient evaluations: 3
